In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import numpy as np
import itertools
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib.pyplot as plt
import os
import glob
import pandas as pd
import pickle
import tensorflow as tf
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input 
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import regularizers
from tensorflow.python.keras.models import load_model
from IPython.display import Audio
plt.style.use('seaborn-white')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


<ipython-input-32-010469583d33>:22: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')


In [81]:
f_voyage = open("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/Ayumi_Hamasaki-VOYAGE.txt", "r")
voy_tmp = f_voyage.read().split("\n")
voy_list = [x.split(",") for x in voy_tmp]
f_secret = open("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/Hikaru_Utada-Can_you_keep_a_secret.txt", "r")
sec_tmp = f_secret.read().split("\n")
sec_list = [x.split(",") for x in sec_tmp]

In [82]:
voy_mp3,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/Voyage.mp3", sr=44100)
sec_mp3,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/Can_You_Keep_A_Secret.mp3", sr=44100)

<ipython-input-82-56434e44b91e>:1: UserWarning: PySoundFile failed. Trying audioread instead.
  voy_mp3,sr = librosa.load("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/Voyage.mp3", sr=44100)
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
display(Audio(data = voy_mp3[:60*44100], rate = 44100))
display(Audio(data = sec_mp3[:10*44100], rate = 44100))

In [ ]:
voy_dict = {"data" : [], "label" : []}
for idx, i in enumerate(voy_list[:-1]):
  diff = -0.35 ## 0.4
  voy_dict["data"].append(voy_mp3[int((float(i[0]) + diff)*44100): int((float(i[1]) + diff)*44100)])
  voy_dict["label"].append(i[2])

voy_df = pd.DataFrame(voy_dict)

for idx in range(len(voy_df)):
  if voy_df.label[idx] == 'vibrato' or voy_df.label[idx] == 'vocalfry':
    print(voy_df.label[idx])
    display(Audio(data = voy_df.data[idx], rate = 44100))

In [ ]:
sec_dict = {"data" : [], "label" : []}

for idx, i in enumerate(sec_list[:-1]):
  diff = 0.25
  sec_dict["data"].append(sec_mp3[int((float(i[0]) + diff)*44100): int((float(i[1]) + diff)*44100)])
  sec_dict["label"].append(i[2])

sec_df = pd.DataFrame(sec_dict)

for idx in range(len(sec_df)):
  if sec_df.label[idx] == 'vibrato' or sec_df.label[idx] == 'vocalfry':
    print(sec_df.label[idx])
    display(Audio(data = sec_df.data[idx], rate = 44100))

vibrato


vibrato


vibrato


vocalfry


vibrato


...

vibrato


vibrato


vocalfry


vibrato


vibrato


In [ ]:
voy_df = voy_df[(voy_df['label'] == 'vibrato') | (voy_df['label'] ==  'vocalfry')]
sec_df = sec_df[(sec_df['label'] == 'vibrato') | (sec_df['label'] ==  'vocalfry')]

In [ ]:
for idx,i in enumerate(voy_df.data):
  if len(i) < 17640:
    voy_df.data.iloc[idx] = np.append(i, [0] * (17640 - len(i)))
  if len(i) > 17640:
    voy_df.data.iloc[idx] = i[:17640]
  if voy_df.label.iloc[idx] == 'vocalfry':
    voy_df.label.iloc[idx] = 'vocal_fry'

<ipython-input-87-3d5f2b38bf50>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voy_df.data.iloc[idx] = np.append(i, [0] * (17640 - len(i)))
<ipython-input-87-3d5f2b38bf50>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voy_df.label.iloc[idx] = 'vocal_fry'
<ipython-input-87-3d5f2b38bf50>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  voy_df.data.iloc[idx] = i[:17640]


In [ ]:
for idx,i in enumerate(sec_df.data):
  if len(i) < 17640:
    sec_df.data.iloc[idx] = np.append(i, [0] * (17640 - len(i)))
  if len(i) > 17640:
    sec_df.data.iloc[idx] = i[:17640]
  if sec_df.label.iloc[idx] == 'vocalfry':
    sec_df.label.iloc[idx] = 'vocal_fry'

<ipython-input-88-82f0c2f4d890>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sec_df.data.iloc[idx] = i[:17640]
<ipython-input-88-82f0c2f4d890>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sec_df.data.iloc[idx] = np.append(i, [0] * (17640 - len(i)))
<ipython-input-88-82f0c2f4d890>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sec_df.label.iloc[idx] = 'vocal_fry'


In [ ]:
voy_df.to_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/voyage_vibfry.pkl")
sec_df.to_pickle("/content/drive/MyDrive/Startup_Hackathon_Jarvis/cosian_data/secret_vibfry.pkl")

In [ ]:
for idx,i in enumerate(voy_df.data):
  print(voy_df.label.iloc[idx])
  display(Audio(data = i, rate = 44100))

vibrato


vibrato


vocal_fry


vibrato


vibrato


...

vibrato


vibrato


vibrato


vibrato


vibrato


In [ ]:
for idx,i in enumerate(sec_df.data):
  print(sedc_df.label.iloc[idx])
  display(Audio(data = i, rate = 44100))

NameError: ignored